In [43]:
import pcl
import rosbag
import numpy as np
import gtsam as gt
import ros_numpy
import sensor_msgs
import time

In [44]:
bag = rosbag.Bag('2022-04-04-18-44-50.bag')

In [45]:
vertex = {} # {time: [i, gt.Pose2]}
i = int(0)
for vertex_str, vertex_msg, vertex_time in bag.read_messages(topics=['/vertex_odom']):
    temp = vertex_msg.data.split(' ')
    t,x,y,theta = np.array(temp[1:]).astype(np.float64)
    temp_pose = gt.Pose2(x, y, theta)
    vertex[t] = [i,temp_pose]
    i = int(i+1)

In [46]:
edges = [] # i,j,x,y,theta,noiseModel
for edge_str, edge_msg, edge_time in bag.read_messages(topics=['/edge_odom']):
    data = edge_msg.data.split(' ')
    st,ct,x,y,theta = np.array(data[1:6]).astype(np.float64)
    q11, q12, q13, q22, q23, q33 = np.array(data[6:]).astype(np.float64)
    info = np.array([q11, q12, q13, q12, q22, q23, q13, q23, q33]).reshape([3,3])
    noiseModel = gt.noiseModel.Gaussian.Information(info)
    if vertex.get(st) is not None and vertex.get(ct) is not None:
        i = vertex.get(st)[0]
        j = vertex.get(ct)[0]
        edges.append([i,j,x,y,theta,noiseModel])

In [47]:
def convert2PCL(pc_msg):
    pc_msg.__class__ = sensor_msgs.msg._PointCloud2.PointCloud2
    offset_sorted = {f.offset: f for f in pc_msg.fields}
    pc_msg.fields = [f for (_, f) in sorted(offset_sorted.items())]
    pc_np = ros_numpy.point_cloud2.pointcloud2_to_xyz_array(pc_msg, remove_nans=True)
    pc_pcl = pcl.PointCloud(np.array(pc_np, dtype=np.float32))
    return pc_pcl

In [48]:
pc_dict = {} # {id: ros::pc2}
for pc_str, pc_msg, pc_time in bag.read_messages(topics=['/currentFeatureInWorld']):
    ct = pc_msg.header.stamp.to_sec()
    idx = vertex.get(ct, vertex[min(vertex.keys(), key=lambda k: abs(k-ct))])[0]  #Find closest index
    pc_dict[idx] = convert2PCL(pc_msg)

In [ ]:
#https://github.com/strawlab/python-pcl/blob/master/examples/official/Registration/iterative_closest_point.py
fitnessThreshold =1.8
interEdges = [] # [i,j,T] 
for idx, inputCloud in pc_dict.items():
    icp = inputCloud.make_IterativeClosestPoint()
    startTime = time.time()
    matchMatrix = [] #[i,fitnessScore,T]
    for i, targetCloud in pc_dict.items():
        if i !=idx :
            boolCoverge, T, estimate, fitnessScore = icp.icp(inputCloud, targetCloud)
            matchMatrix.append([i,fitnessScore,T])
    matchMatrix = np.array(matchMatrix)
    bestMatches = matchMatrix[matchMatrix[:,1]>fitnessThreshold]
    for bM in bestMatches:
        interEdges.append([idx, bM[0], bM[2]]) # [i,j,T]   
    print(time.time()-startTime)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Non length-one string passed in for the array ordering. Please pass in 'C', 'F', 'A', or 'K' instead
  if __name__ == "__main__":


6.8624491691589355
6.847267150878906
8.711354970932007
8.683400869369507
9.313524007797241
8.48064923286438
8.441345691680908
7.613246440887451
8.385488510131836
8.299804925918579
6.755072832107544
7.670130968093872
8.72962760925293
9.930936813354492
7.874687910079956
8.935747861862183
8.125187873840332
8.104053020477295
8.11383843421936
8.549787044525146
8.283461809158325
7.985477924346924
8.786146640777588
8.448326110839844
7.434619665145874
8.166936159133911
8.058042287826538
7.575127363204956
8.273138284683228
8.639071464538574
7.8889970779418945
9.056118965148926
8.253843545913696
9.2789785861969
8.016488313674927
8.415921926498413
8.51584792137146
8.131929397583008
8.631628513336182
8.55404281616211
8.382139682769775
8.793489694595337
9.342810153961182
9.380035400390625
8.917288541793823
9.238921403884888
9.49570107460022
9.554077863693237
9.331871747970581
9.22998595237732
9.22432255744934
9.266549587249756
9.417516708374023
9.65219259262085
9.1229407787323
9.24932599067688
9.02

In [ ]:
np.savez('processedData.npz', vertex=**vertex, edges=np.array(edges), interEdges=np.array(interEdges))